In [1]:
import numpy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim

from PIL import Image
from torch.utils.data.dataset import Dataset
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torchvision import transforms, datasets
import matplotlib.pyplot as plt

In [2]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

In [7]:
transf = transforms.Compose([transforms.Grayscale(),transforms.ToTensor()])
dataset = datasets.ImageFolder('PPC2/.',transf)
n = len(dataset)

ntrain = int(0.8 *n)
ntest = n - ntrain
train_data, test_data = torch.utils.data.random_split(dataset, [ntrain, ntest])
nbatch = 8
print (ntrain)
print (ntest)
print (n)

161
41
202


In [8]:
dataload_train = dict(shuffle=True, batch_size=nbatch,num_workers=4, pin_memory=True)
train_load = dataloader.DataLoader(train_data, **dataload_train)

dataload_test = dict(shuffle=True, batch_size=ntest,num_workers=4, pin_memory=True)
test_load = dataloader.DataLoader(test_data, **dataload_test)

dataiter = iter(test_load)
images, labels = dataiter.next()

In [9]:
# One hidden Layer NN
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc0 = nn.Linear(32 * 32, 64*64)
        self.fc1 = nn.Linear(64 * 64, 16*16)
        self.fcout = nn.Linear(16*16,3)
    def forward(self, x):
        out = self.fc0(x.view(-1,32*32))
        out = F.sigmoid(out)
        out = F.sigmoid(self.fc1(out))
        out = F.relu(self.fcout(out))
        return F.log_softmax(out, dim=0)    
    
    
model = Model()
if cuda:
    model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 20
losses = []


In [13]:
model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_load):
        # Get Samples
        data, target = Variable(data), Variable(target)
        
        if cuda:
            data, target = data.cuda(), target.cuda()

        # Init
        optimizer.zero_grad()
        # Predict
        y_pred = model(data) 

         
        # Calculate loss
        #print(y_pred)
        #print(target)
        loss = F.cross_entropy(y_pred, target)
        losses.append(loss.cpu().item())
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        
        # Display
        if batch_idx % 100 == 1:
            print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                EPOCHS,
                batch_idx * len(data), 
                ntrain,
                100. * batch_idx / nbatch, 
                loss.cpu().item()), 
                end='')
            
    # display final evaluation for this epoch
    model.eval()
    output = model(images)
    pred = output.data.max(1)[1]
    d = pred.eq(labels.data).cpu()
    accuracy = d.sum().item()/d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Test Accuracy: {:.4f}%'.format(
        epoch+1,
        EPOCHS,
        ntrain, 
        ntrain,
        100. * batch_idx / nbatch, 
        loss.cpu().item(),
        accuracy*100,
        end=''))

/home/boukraichi/.local/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


 Train Epoch: 1/20 [161/161 (12%)]	Loss: 1.098612	 Test Accuracy: 36.5854%
 Train Epoch: 2/20 [161/161 (12%)]	Loss: 1.098612	 Test Accuracy: 36.5854%
 Train Epoch: 3/20 [161/161 (12%)]	Loss: 1.098612	 Test Accuracy: 36.5854%
 Train Epoch: 4/20 [161/161 (12%)]	Loss: 1.098612	 Test Accuracy: 36.5854%
 Train Epoch: 5/20 [161/161 (12%)]	Loss: 1.098612	 Test Accuracy: 36.5854%
 Train Epoch: 6/20 [161/161 (12%)]	Loss: 1.098612	 Test Accuracy: 36.5854%
 Train Epoch: 7/20 [161/161 (12%)]	Loss: 1.098612	 Test Accuracy: 36.5854%
 Train Epoch: 8/20 [161/161 (12%)]	Loss: 1.098612	 Test Accuracy: 36.5854%
 Train Epoch: 9/20 [161/161 (12%)]	Loss: 1.098612	 Test Accuracy: 36.5854%
 Train Epoch: 10/20 [161/161 (12%)]	Loss: 1.098612	 Test Accuracy: 36.5854%
 Train Epoch: 11/20 [161/161 (12%)]	Loss: 1.098612	 Test Accuracy: 36.5854%
 Train Epoch: 12/20 [161/161 (12%)]	Loss: 1.098612	 Test Accuracy: 36.5854%
 Train Epoch: 13/20 [161/161 (12%)]	Loss: 1.098612	 Test Accuracy: 36.5854%
 Train Epoch: 14/20 [